# NYC Open Data Downloader

This notebook downloads 911 data from NYC Open Data API. It retrieves records for the year 2024 with specific radio codes, handling the data in batches to manage memory efficiently.

## Import Required Libraries
We'll need the following libraries:
- `requests`: For making HTTP requests
- `json`: For handling JSON data
- `gzip`: For compression
- `datetime`: For timestamp handling
- `pandas`: For data manipulation
- `time`: For adding delays between requests
- `dotenv`: For loading environment variables from .env file
- `os`: For environment variables
- `pathlib`: For handling file paths

In [1]:
import requests
import json
import gzip
from datetime import datetime
import pandas as pd
from time import sleep
from dotenv import load_dotenv
import os
from pathlib import Path

## API Configuration
Set up the API endpoint and authentication headers using credentials from a .env file. We'll also define our pagination parameters.

Note: Make sure you have a .env file in the same directory with your `NYC_DATA_AUTH` value.

In [2]:
url = 'https://data.cityofnewyork.us/api/v3/views/d6zx-ckhd/query.json'

# Load environment variables from .env file
env_path = Path().absolute() / '.env'
load_dotenv(dotenv_path=env_path)

# Get authorization from .env file
auth_token = os.getenv('NYC_DATA_AUTH')
if not auth_token:
    raise ValueError(".env file missing NYC_DATA_AUTH value")

HEADERS = {
    'Content-Type': 'application/json',
    'Authorization': f'Basic {auth_token}'
}

all_data = []
page_number = 1
page_size = 50000

## Data Download Loop
This section contains the main loop that:
1. Constructs the query payload
2. Makes API requests
3. Handles pagination
4. Saves intermediate results
5. Implements error handling and retries

We will first run the loop to extract the data for radio code that correspond to criminal activites (//to do attach link to file with mapping of radio code to categories)

In [3]:
while True:
    payload = {
        "query": "SELECT cad_evnt_id, create_date,incident_date,incident_time,nypd_pct_cd,boro_nm,patrl_boro_nm,geo_cd_x,geo_cd_y,radio_code,typ_desc,cip_jobs, add_ts, arrivd_ts, disp_ts,closng_ts,latitude,longitude  WHERE incident_date between '2024-01-01' and '2024-12-31' AND boro_nm IS NOT NULL AND patrl_boro_nm IS NOT NULL AND nypd_pct_cd IS NOT NULL AND closng_ts IS NOT NULL AND radio_code IS NOT NULL AND typ_desc IS NOT NULL AND cip_jobs IS NOT NULL AND disp_ts IS NOT NULL AND arrivd_ts IS NOT NULL AND radio_code in ('34K7', '34K6', '34K1', '34K8', '34K2', '34K5', '34K9', '34Q7', '34Q6', '34Q1', '34Q8', '34Q2', '34Q5', '34Q9', '34S7', '34S6', '34S1', '34S8', '34S2', '34S5', '34S9', '34W7', '34W6', '34W1', '34W8', '34W2', '34W5', '34W9', '24K7', '24K6', '24K1', '24K8', '24K2', '24K5', '24K9', '24Q7', '24Q6', '24Q1', '24Q8', '24Q2', '24Q5', '24Q9', '24S7', '24S6', '24S1', '24S2', '24S5', '24S9', '24W7', '24W6', '24W1', '24W8', '24W2', '24W5', '24W9', '31C', '31C9', '31Q', '31C8', '31Q8', '31Q5', '31Q9', '31R', '21C', '21C9', '21Q', '21Q5', '21Q9', '21R', '50G1', '50G8', '50G2', '50G5', '50G9', '50N1', '50N8', '50N2', '50N9', '50P1', '50P8', '50P2', '50P5', '50P9', '32P1', '32P8', '32P2', '32P5', '32P9', '32Q1', '32Q8', '32Q2', '32Q5', '32Q9', '32V1', '32V8', '32V2', '32V9', '22P1', '22P8', '22P2', '22P5', '22P9', '22Q1', '22Q8', '22Q2', '22Q5', '22Q9', '22V1', '22V8', '22V2', '22V5', '22V9', '39C1', '39C8', '39C2', '39C5', '39C9', '39P6', '39G8', '39G9', '39G', '39H1', '39H8', '39H2', '39H5', '39H9', '39Q1', '39Q8', '39Q2', '39Q5', '39Q9', '39P', '39T1', '39T8', '39T2', '39T5', '39T9', '39V6', '39V1', '39V8', '39V2', '39V5', '39V9', '29C8', '29C9', '29C', '29B9', '29H1', '29H8', '29H2', '29H5', '29H9', '29Q1', '29Q8', '29Q2', '29Q5', '29Q9', '29T1', '29T2', '29T5', '29T9', '69M1', '69M2', '69M9', '30B', '30C', '30C8', '30C9', '30Q1', '30Q8', '30Q2', '30Q5', '30Q9', '30R', '20B', '20C', '20Q1', '20Q8', '20Q2', '20Q5', '20Q9', '20R')",
        "page": {
            "pageNumber": page_number,
            "pageSize": page_size
        },
        "includeSynthetic": False
    }
    
    print(f"Fetching page {page_number}... Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
    try:
        response = requests.post(url, headers=HEADERS, json=payload, timeout=300)
        response.raise_for_status()
    except requests.exceptions.Timeout:
        print(f"Timeout error on page {page_number}. Retrying...")
        sleep(5)
        continue
    except requests.exceptions.RequestException as e:
        print(f"Error on page {page_number}: {e}")
        break
    
    # Check if request was successful
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        break
    
    data = response.json()
    
    # Check if we got any data
    if not data or len(data) == 0:
        print("No more data to fetch.")
        break
    
    all_data.extend(data)
    print(f"Retrieved {len(data)} records. Total so far: {len(all_data)}")
    
    # If we got fewer records than page_size, we've reached the end
    if len(data) < page_size:
        print("Reached the last page.")
        break
    
    page_number += 1
    sleep(1)  # Add a 1-second delay between requests

Fetching page 1... Timestamp: 2025-11-01 19:57:11
Retrieved 50000 records. Total so far: 50000
Fetching page 2... Timestamp: 2025-11-01 19:57:21
Retrieved 50000 records. Total so far: 100000
Fetching page 3... Timestamp: 2025-11-01 19:58:38
Retrieved 50000 records. Total so far: 150000
Fetching page 4... Timestamp: 2025-11-01 19:58:45
Retrieved 50000 records. Total so far: 200000
Fetching page 5... Timestamp: 2025-11-01 19:58:53
Retrieved 50000 records. Total so far: 250000
Fetching page 6... Timestamp: 2025-11-01 19:59:00
Retrieved 50000 records. Total so far: 300000
Fetching page 7... Timestamp: 2025-11-01 19:59:06
Retrieved 50000 records. Total so far: 350000
Fetching page 8... Timestamp: 2025-11-01 19:59:13
Retrieved 50000 records. Total so far: 400000
Fetching page 9... Timestamp: 2025-11-01 19:59:19
Retrieved 50000 records. Total so far: 450000
Fetching page 10... Timestamp: 2025-11-01 19:59:26
Retrieved 50000 records. Total so far: 500000
Fetching page 11... Timestamp: 2025-11-0

## Save Final Results
Save the complete dataset to a compressed JSON file.

In [4]:
print(f"\nTotal records extracted: {len(all_data)}")

# Save to JSON file
output_filename = 'nyc_data_crime.json.gz'
with gzip.open(output_filename, 'wt', encoding='utf-8') as f:
    json.dump(all_data, f, ensure_ascii=False, separators=(',', ':'))

print(f"Data saved to {output_filename}")


Total records extracted: 815867
Data saved to nyc_data_crime.json.gz


Save the complete dataset to a compressed parquet file.

In [5]:
# Convert to DataFrame
df = pd.DataFrame(all_data)

# Save as Parquet with zstd compression
output_filename = 'nyc_data_crime.parquet'
df.to_parquet(output_filename, compression='zstd', index=False)

print(f"Data saved to {output_filename}")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")


Data saved to nyc_data_crime.parquet
Shape: (815867, 18)
Columns: ['cad_evnt_id', 'create_date', 'incident_date', 'incident_time', 'nypd_pct_cd', 'boro_nm', 'patrl_boro_nm', 'geo_cd_x', 'geo_cd_y', 'radio_code', 'typ_desc', 'cip_jobs', 'add_ts', 'arrivd_ts', 'disp_ts', 'closng_ts', 'latitude', 'longitude']


Now downloading data for potential-criminal activities

In [6]:
all_data = []
page_number = 1
page_size = 50000
while True:
    payload = {
        "query": "SELECT cad_evnt_id, create_date,incident_date,incident_time,nypd_pct_cd,boro_nm,patrl_boro_nm,geo_cd_x,geo_cd_y,radio_code,typ_desc,cip_jobs, add_ts, arrivd_ts, disp_ts,closng_ts,latitude,longitude  WHERE incident_date between '2024-01-01' and '2024-12-31' AND boro_nm IS NOT NULL AND patrl_boro_nm IS NOT NULL AND nypd_pct_cd IS NOT NULL AND closng_ts IS NOT NULL AND radio_code IS NOT NULL AND typ_desc IS NOT NULL AND cip_jobs IS NOT NULL AND disp_ts IS NOT NULL AND arrivd_ts IS NOT NULL AND radio_code in ('11A1', '11A2', '11A5', '11A9', '11B4', '11B3', '11B9', '11C4', '11C3', '11C8', '11Q4', '11Q3', '11Q9', '11R3', '11R4', '52D6', '52F6', '52F1', '52F8', '52F2', '52F5', '52F9', '52D1', '52K6', '52K1', '52K8', '52K2', '52K5', '52K9', '52D8', '52D2', '52A6', '52D5', '52D9', '52V6', '52V1', '52V2', '52V5', '52V9', '33', '33S5', '33T9', '10H1', '10H8', '10H2', '10H5', '10H9', '10F1', '10F8', '10F2', '10F5', '10F9', '10K1', '10K8', '10K2', '10K5', '10K9', '10M1', '10M2', '10M5', '10M9', '10N1', '10N8', '10N2', '10N5', '10N9', '10Q1', '10Q8', '10Q2', '10Y5', '10Q5', '10Q9', '10Y7', '10Y8', '10Y9', '10Y3', '10S1', '10S2', '10S5', '10S9', '10S8', '10P1', '10P8', '10P2', '10P5', '10P9', '10V1', '10V8', '10V2', '10V9', '40V', '40V8', '40V5', '12Z1', '12Z2', '12U1', '12U2', '12U5', '12U9', '12X', '12X1', '12X8', '12X2', '12X5', '12X9', '71S', '10S4', '44L1', '44L2', '44L5', '44P1', '44P8', '44P2', '44P5', '44P9', '44S1', '44S2', '44S9')",
        "page": {
            "pageNumber": page_number,
            "pageSize": page_size
        },
        "includeSynthetic": False
    }
    
    print(f"Fetching page {page_number}... Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
    try:
        response = requests.post(url, headers=HEADERS, json=payload, timeout=300)
        response.raise_for_status()
    except requests.exceptions.Timeout:
        print(f"Timeout error on page {page_number}. Retrying...")
        sleep(5)
        continue
    except requests.exceptions.RequestException as e:
        print(f"Error on page {page_number}: {e}")
        break
    
    # Check if request was successful
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        break
    
    data = response.json()
    
    # Check if we got any data
    if not data or len(data) == 0:
        print("No more data to fetch.")
        break
    
    all_data.extend(data)
    print(f"Retrieved {len(data)} records. Total so far: {len(all_data)}")
    
    # If we got fewer records than page_size, we've reached the end
    if len(data) < page_size:
        print("Reached the last page.")
        break
    
    page_number += 1
    sleep(1)  # Add a 1-second delay between requests

Fetching page 1... Timestamp: 2025-11-01 20:08:23
Retrieved 50000 records. Total so far: 50000
Fetching page 2... Timestamp: 2025-11-01 20:08:46
Timeout error on page 2. Retrying...
Fetching page 2... Timestamp: 2025-11-01 20:13:51
Retrieved 50000 records. Total so far: 100000
Fetching page 3... Timestamp: 2025-11-01 20:14:39
Retrieved 50000 records. Total so far: 150000
Fetching page 4... Timestamp: 2025-11-01 20:14:53
Retrieved 50000 records. Total so far: 200000
Fetching page 5... Timestamp: 2025-11-01 20:15:02
Retrieved 50000 records. Total so far: 250000
Fetching page 6... Timestamp: 2025-11-01 20:15:13
Retrieved 50000 records. Total so far: 300000
Fetching page 7... Timestamp: 2025-11-01 20:15:23
Retrieved 50000 records. Total so far: 350000
Fetching page 8... Timestamp: 2025-11-01 20:15:34
Retrieved 50000 records. Total so far: 400000
Fetching page 9... Timestamp: 2025-11-01 20:15:46
Retrieved 50000 records. Total so far: 450000
Fetching page 10... Timestamp: 2025-11-01 20:15:56

In [7]:
print(f"\nTotal records extracted: {len(all_data)}")

# Save to JSON file
output_filename = 'nyc_data_potential_crime.json.gz'
with gzip.open(output_filename, 'wt', encoding='utf-8') as f:
    json.dump(all_data, f, ensure_ascii=False, separators=(',', ':'))

print(f"Data saved to {output_filename}")


Total records extracted: 1117591
Data saved to nyc_data_potential_crime.json.gz


In [8]:
# Convert to DataFrame
df = pd.DataFrame(all_data)

# Save as Parquet with zstd compression
output_filename = 'nyc_data_potential_crime.parquet'
df.to_parquet(output_filename, compression='zstd', index=False)

print(f"Data saved to {output_filename}")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")


Data saved to nyc_data_potential_crime.parquet
Shape: (1117591, 18)
Columns: ['cad_evnt_id', 'create_date', 'incident_date', 'incident_time', 'nypd_pct_cd', 'boro_nm', 'patrl_boro_nm', 'geo_cd_x', 'geo_cd_y', 'radio_code', 'typ_desc', 'cip_jobs', 'add_ts', 'arrivd_ts', 'disp_ts', 'closng_ts', 'latitude', 'longitude']
